In [344]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import altair as alt
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

### Load Data

In [345]:
df = pd.read_csv("fashion3.csv")
df.head()

,Unnamed: 0.1,Unnamed: 0,item_id,category,waist,size,quality,cup size,hips,bra size,height_inches,length,fit,review_summary,review_text,polarity,reviewLen,wordCount
0,0,8490,159891,1,26.0,1,5.0,2.0,38.0,32.0,64.0,5.0,3,This dress. Is so amazing,This dress. Is so amazing. It just arrived tod...,0.392500,301,58
1,1,8493,159891,1,31.0,8,3.0,5.0,41.0,36.0,62.0,5.0,3,This dress looks great on,This dress looks great on. I'm a pretty curren...,0.186250,273,57
2,2,8521,160625,1,32.0,5,5.0,2.0,NaN,32.0,65.0,5.0,1,Used as my wedding dress,Used as my wedding dress and I LOVED it!PROS ...,0.172436,452,85
3,3,8523,160625,1,32.0,11,4.0,5.0,43.0,38.0,67.0,5.0,2,"I love this dress, it's b","I love this dress, it's beautiful ! Wore this ...",0.413839,581,125
4,4,8524,160625,1,28.0,7,5.0,3.0,32.0,34.0,65.0,5.0,3,I love this dress!!! I bo,I love this dress!!! I bought it specifically ...,0.400769,627,117


In [346]:
df.iloc[:,2:13]
col_lst = list(range(2,13))
col_lst.extend([15,17])
df = df.iloc[:,col_lst]
df_clean = df.dropna()

In [347]:
df_clean.head()

,item_id,category,waist,size,quality,cup size,hips,bra size,height_inches,length,fit,polarity,wordCount
0,159891,1,26.0,1,5.0,2.0,38.0,32.0,64.0,5.0,3,0.392500,58
1,159891,1,31.0,8,3.0,5.0,41.0,36.0,62.0,5.0,3,0.186250,57
3,160625,1,32.0,11,4.0,5.0,43.0,38.0,67.0,5.0,2,0.413839,125
4,160625,1,28.0,7,5.0,3.0,32.0,34.0,65.0,5.0,3,0.400769,117
7,160625,1,27.0,5,5.0,3.0,38.0,36.0,68.0,5.0,2,0.098889,42


# SVD

User's rating scores:

* `quality`, and `polarity`

Because both features have different scaling, we need to make sure that they have same scaling to combine them together

In [348]:
# Scaling range: [0,1]
# After scaling, take the average to get the overall score
col_lst = [4,11]
score = df_clean.iloc[:,col_lst].apply(lambda iterator: ((iterator - iterator.min())/(iterator.max() - iterator.min())).round(2))
score["avg"] = score.mean(axis=1)
score.head()

,quality,polarity,avg
0,1.00,0.69,0.845
1,0.50,0.59,0.545
3,0.75,0.70,0.725
4,1.00,0.70,0.850
7,1.00,0.54,0.770


In [349]:
 # Add user id and category
score.insert(0, "user_id", score.index + 1)
score.insert(1, "item_id", df_clean["category"])

In [350]:
# only keep the first three columns
score = score.iloc[:,0:3]
score.head()

,user_id,item_id,quality
0,1,1,1.00
1,2,1,0.50
3,4,1,0.75
4,5,1,1.00
7,8,1,1.00


In [351]:
score2 = score["quality"]

In [352]:
score = score.pivot(index='user_id', columns='item_id', values='quality') 

In [353]:
score.head()

item_id,1,2,3,4
user_id,,,,
1,1.00,NaN,NaN,NaN
2,0.50,NaN,NaN,NaN
4,0.75,NaN,NaN,NaN
5,1.00,NaN,NaN,NaN
8,1.00,NaN,NaN,NaN


In [354]:
df_top = df_clean[['waist','height_inches','cup size', 'bra size','fit']].copy()

In [355]:
score.fillna(0)

item_id,1,2,3,4
user_id,,,,
1,1.00,0.0,0.0,0.00
2,0.50,0.0,0.0,0.00
4,0.75,0.0,0.0,0.00
5,1.00,0.0,0.0,0.00
8,1.00,0.0,0.0,0.00
...,...,...,...,...
2090,0.00,0.0,0.0,0.00
2091,0.00,0.0,0.0,1.00
2094,0.00,0.0,0.0,1.00


In [356]:
X = df_top
y = score2

In [357]:
scaler = StandardScaler()

In [358]:
scaler.fit(X)

StandardScaler()

In [359]:
X_scaled = scaler.transform(X)

In [360]:
reg = KNeighborsRegressor(n_neighbors=15)

In [361]:
reg.fit(X_scaled,y)

KNeighborsRegressor(n_neighbors=15)

In [362]:
score["top pred"] = reg.predict(X_scaled)

In [363]:
score

item_id,1,2,3,4,top pred
user_id,,,,,
1,1.00,NaN,NaN,NaN,0.783333
2,0.50,NaN,NaN,NaN,0.783333
4,0.75,NaN,NaN,NaN,0.700000
5,1.00,NaN,NaN,NaN,0.933333
8,1.00,NaN,NaN,NaN,0.666667
...,...,...,...,...,...
2090,NaN,NaN,NaN,0.00,0.700000
2091,NaN,NaN,NaN,1.00,0.800000
2094,NaN,NaN,NaN,1.00,0.800000


In [364]:
X_scaled_train, X_scaled_test, y_train, y_test = train_test_split(X_scaled,y,test_size=0.2)

In [365]:
reg.fit(X_scaled_train,y_train)

KNeighborsRegressor(n_neighbors=15)

In [366]:
mean_squared_error(reg.predict(X_scaled_test),y_test)

0.06950171821305842

In [367]:
mean_squared_error(reg.predict(X_scaled_train),y_train)

0.051142850324551355

In [368]:
def get_scores(k):
    reg = KNeighborsRegressor(n_neighbors=k)
    reg.fit(X_scaled_train, y_train)
    train_error = mean_absolute_error(reg.predict(X_scaled_train), y_train)
    test_error = mean_absolute_error(reg.predict(X_scaled_test), y_test)
    return (train_error, test_error)

df_scores = pd.DataFrame({"k":range(1,150),"train_error":np.nan,"test_error":np.nan})

for i in df_scores.index:
    df_scores.loc[i,["train_error","test_error"]] = get_scores(df_scores.loc[i,"k"])

df_scores["kinv"] = 1/df_scores.k

In [369]:
ctrain = alt.Chart(df_scores).mark_line(color="#FF0000").encode(
    x = "kinv",
    y = "train_error"
)

In [370]:
ctest = alt.Chart(df_scores).mark_line(color="#045FB4").encode(
    x = alt.X("kinv",title='k-inverse'),
    y = "test_error"
)

In [371]:
# plot for mean squared error for tops
ctrain+ctest

alt.LayerChart(...)

In [372]:
df_bottoms = df_clean[['waist','height_inches','fit']].copy()

In [373]:
Z = df_bottoms

In [374]:
scaler.fit(Z)

StandardScaler()

In [375]:
Z_scaled = scaler.transform(Z)

In [376]:
Z_scaled_train, Z_scaled_test, y_train, y_test = train_test_split(Z_scaled,y,test_size=0.2)

In [377]:
reg.fit(Z_scaled,y)

KNeighborsRegressor(n_neighbors=15)

In [378]:
mean_squared_error(reg.predict(Z_scaled_test),y_test)

0.05345838106147384

In [379]:
mean_squared_error(reg.predict(Z_scaled_train),y_train)

0.05175663421153112

In [380]:
def get_scores_b(k):
    reg = KNeighborsRegressor(n_neighbors=k)
    reg.fit(Z_scaled_train, y_train)
    train_error_b = mean_absolute_error(reg.predict(Z_scaled_train), y_train)
    test_error_b = mean_absolute_error(reg.predict(Z_scaled_test), y_test)
    return (train_error_b, test_error_b)

df_scores_b = pd.DataFrame({"k":range(1,150),"train_error_b":np.nan,"test_error_b":np.nan})

for i in df_scores.index:
    df_scores_b.loc[i,["train_error_b","test_error_b"]] = get_scores_b(df_scores_b.loc[i,"k"])

df_scores_b["kinv"] = 1/df_scores_b.k

In [381]:
ctrain_b = alt.Chart(df_scores).mark_line(color="#FF0000").encode(
    x = "kinv",
    y = "train_error"
)

In [382]:
ctest_b = alt.Chart(df_scores).mark_line(color="#045FB4").encode(
    x = alt.X("kinv",title='k-inverse'),
    y = "test_error"
)

In [383]:
ctrain_b+ctest_b

alt.LayerChart(...)

In [384]:
df_ow = df_clean[['waist','height_inches','cup size', 'bra size','fit']].copy()